In [1]:
#imports
import pyodbc
import pandas as pd

In [19]:
#connect to db
driver = "{ODBC Driver 17 for SQL Server}"
server = "172.20.10.149\PRODUCTION"
database = "stb_production"
username = "Neo.Tech"
password = "Password357"

conn = pyodbc.connect("DRIVER=" + driver
+ ";SERVER=" + server
+ ";DATABASE=" + database
+ ";UID=" + username
+ ";PWD=" + password )
cursor = conn.cursor()

In [62]:
filename = 'BGA Rework - 3136 Units.xlsx'

pallet_col = 'Pallet 1'


excel_data = pd.read_excel('BGA Rework - 3136 Units.xlsx')
# Read the values of the file in the dataframe
df = pd.DataFrame(excel_data, columns=[pallet_col])
# Print the content
# print("The content of the file is:\n", df)

In [68]:
stb_find_str = '\''
stb_find_list = []
# {
#     search: '',
#     found: '',
#     iuc: '',
#     pcb: ''
# }
product = '4138'

pallet_stbs = df.iterrows()

# print(df.to_numpy().flatten())
pallet_stbs_list = df.to_numpy().flatten()

print(f'{pallet_col} has {len(pallet_stbs_list)} stbs')

## TEST PARAMS
test_iter = 7

for stb_num_short in pallet_stbs_list:
    print(f'--iter-- {test_iter}')
    temp = {
        "search": stb_num_short,
        "found": '',
        "iuc": '',
        "pcb": ''
    }
    if (test_iter >= 0):
        test_iter -=1
    else:
        break
    print(f'{stb_num_short}')
    # time.sleep(1)
    for row in cursor.execute(f'''SELECT stb_num FROM stb_production.dbo.production_event
    WHERE stb_num LIKE \'{stb_num_short}%\''''):
        # print(f'>> {row.stb_num}')
        stb_find_str = stb_find_str + row.stb_num + '\',\''
        # stb_find_list.append(row.stb_num)
        temp['found'] = row.stb_num.strip()
        print(f'>> {stb_find_str}')
    stb_find_list.append(temp)

# Removes last character from string
stb_find_str = stb_find_str[:-2]

Pallet 1 has 784 stbs
--iter-- 7
S041153204
--iter-- 6
S040789152
--iter-- 5
S040006732
>> 'S0400067325','
--iter-- 4
S040846160
--iter-- 3
S041400465
>> 'S0400067325','S0414004652','
--iter-- 2
S041170035
--iter-- 1
S040864251
--iter-- 0
S040977392
>> 'S0400067325','S0414004652','S0409773923','
--iter-- -1


In [78]:
print(f' SQL Input String for Like Statement {stb_find_str}')
# print(f' SQL Input List Like Statement {stb_find_list}')
# for row in cursor.execute(f'''SELECT stb_num FROM stb_production.dbo.production_event
# WHERE stb_num LIKE \'{input_csv_stb_num}%\''''):
#     print(row.stb_num)

# TODO: improve, merge all above stb_nums with ''
print(f'''{"KEY_WORD"} {"STB_NUM"}\t\t\t{"IUC_NUM"}\t\t\t{"PCB_NUM"}''')
for search_obj in stb_find_list:
    # print(search_obj)
    if (search_obj['found'] != ''):
        for innerrow in cursor.execute(f'''SELECT stb_num, cdsn_iuc, pcb_num FROM stb_production.dbo.production_event
                                    INNER JOIN
                                    device_state_dsd_{product} ON production_event.id_production_event = device_state_dsd_{product}.id_production_event
                                    WHERE        stb_num IN ({stb_find_str})'''):
                                    if(innerrow.stb_num.startswith(search_obj['found'])):
                                        print(f'''{search_obj['search']}\t\t{innerrow.stb_num}\t\t{innerrow.cdsn_iuc}\t\t{innerrow.pcb_num}''')
                                    else:
                                        print(f'''{'ERROR'}\t\t{'ERROR'}\t\t{'ERROR'}''')
    else:
        print(f'''{search_obj['search']}\t\t{'NOT FOUND'}\t\t{'NOT FOUND'}\t\t{'NOT FOUND'}''')



 SQL Input String for Like Statement 'S0400067325','S0414004652','S0409773923'
KEY_WORD STB_NUM			IUC_NUM			PCB_NUM
S041153204		NOT FOUND		NOT FOUND		NOT FOUND
S040789152		NOT FOUND		NOT FOUND		NOT FOUND
S040006732		S0400067325		7030203093		308473008871
ERROR		ERROR		ERROR
ERROR		ERROR		ERROR
S040846160		NOT FOUND		NOT FOUND		NOT FOUND
ERROR		ERROR		ERROR
ERROR		ERROR		ERROR
S041400465		S0414004652		7042585573		308953036166
S041170035		NOT FOUND		NOT FOUND		NOT FOUND
S040864251		NOT FOUND		NOT FOUND		NOT FOUND
ERROR		ERROR		ERROR
S040977392		S0409773923		7036343833		308781016256
ERROR		ERROR		ERROR
